In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob

In [52]:
import sys
import os
# Add src directory to the Python path
system_path = os.path.abspath('..')
if system_path in sys.path:
    print('Path already exist')
else:
    sys.path.append(system_path)



Path already exist


In [53]:
%load_ext autoreload
%autoreload 2
from src.EDA_publisher_analysis import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\W-HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
apple_data = pd.read_csv('../data/AAPL_historical_data.csv')
amazon_data = pd.read_csv('../data/AMZN_historical_data.csv')
google_data = pd.read_csv('../data/GOOG_historical_data.csv')
meta_data = pd.read_csv('../data/META_historical_data.csv')
microsoft_data = pd.read_csv('../data/MSFT_historical_data.csv')
nividia_data = pd.read_csv('../data/NVDA_historical_data.csv')
tesla_data = pd.read_csv('../data/TSLA_historical_data.csv')

news_df = pd.read_csv('../data/raw_analyst_ratings.csv')

In [ ]:
# Add a 'Company' column to each dataframe
apple_data['Company'] = 'Apple'
amazon_data['Company'] = 'Amazon'
google_data['Company'] = 'Google'
meta_data['Company'] = 'Meta'
microsoft_data['Company'] = 'Microsoft'
nividia_data['Company'] = 'Nvidia'
tesla_data['Company'] = 'Tesla'

# Concatenate the dataframes
stock_df = pd.concat([apple_data, amazon_data, google_data, meta_data, microsoft_data, nividia_data, tesla_data])

In [57]:
news_df.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [58]:
stock_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Company
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.098943,469033600,0.0,0.0,Apple
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093781,175884800,0.0,0.0,Apple
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086898,105728000,0.0,0.0,Apple
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089049,86441600,0.0,0.0,Apple
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091630,73449600,0.0,0.0,Apple


In [62]:
news_df.shape, stock_df.shape

((1048575, 5), (45428, 10))

In [63]:
news_df= news_df.drop(news_df.columns[0], axis=1)

In [64]:
# Convert 'date' columns to datetime
news_df['date'] = pd.to_datetime(news_df['date'],  errors='coerce').dt.tz_localize(None)
stock_df['Date'] = pd.to_datetime(stock_df['Date'],  errors='coerce').dt.tz_localize(None)

In [65]:
## Checking formatting
news_df.head()

,url,publisher,date,stock
0,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54,A
1,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20,A
2,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07,A
3,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06,A
4,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59,A


In [66]:
news_df.head()

,url,publisher,date,stock
0,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54,A
1,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20,A
2,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07,A
3,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06,A
4,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59,A


In [67]:
# Rename 'Date' column to 'date' in stock_df and convert to datetime format
stock_df.rename(columns={'Date': 'date'}, inplace=True)

In [68]:
stock_df.head()

,date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Company
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.098943,469033600,0.0,0.0,Apple
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093781,175884800,0.0,0.0,Apple
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086898,105728000,0.0,0.0,Apple
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089049,86441600,0.0,0.0,Apple
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091630,73449600,0.0,0.0,Apple


In [69]:
news_df['date'] = pd.to_datetime(news_df['date']).dt.date
stock_df['date'] = pd.to_datetime(stock_df['date']).dt.date

In [70]:
# Merge the two DataFrames on the 'date' column
merged_df = pd.merge(news_df, stock_df, on='date')
merged_df.head()

,url,publisher,date,stock,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Company
0,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,80.837502,82.937500,80.807503,82.875000,80.843407,137250400,0.0,0.0,Apple
1,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,122.225502,124.432503,121.856499,124.150002,124.150002,66128000,0.0,0.0,Amazon
2,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,70.658501,72.252502,70.300003,71.919502,71.837753,34698000,0.0,0.0,Google
3,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,226.710007,231.350006,225.309998,230.770004,230.296753,16750400,0.0,0.0,Meta
4,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,182.619995,187.729996,182.009995,187.199997,180.403564,39893600,0.0,0.0,Microsoft


In [71]:
merged_df.isnull().sum()

url             0
publisher       0
date            0
stock           0
Open            0
High            0
Low             0
Close           0
Adj Close       0
Volume          0
Dividends       0
Stock Splits    0
Company         0
dtype: int64

In [72]:
merged_df.duplicated().sum()

np.int64(0)

In [73]:
merged_df.describe()

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
count,285028.000000,285028.000000,285028.000000,285028.000000,285028.000000,2.850280e+05,285028.000000,285028.000000
mean,76.112213,77.277876,75.071377,76.210124,74.925382,1.551295e+08,0.001723,0.000126
std,62.452522,63.369883,61.634852,62.555049,61.797186,2.026702e+08,0.025837,0.023993
min,0.287250,0.290750,0.278750,0.284500,0.260951,1.584340e+05,0.000000,0.000000
25%,24.667500,25.059999,24.080000,24.665001,23.302501,3.241000e+07,0.000000,0.000000
50%,61.566666,62.944500,60.610668,61.232498,60.253620,7.587200e+07,0.000000,0.000000
75%,116.472000,117.570000,114.209999,115.799500,115.704002,1.896200e+08,0.000000,0.000000
max,240.960007,241.210007,235.279999,238.669998,238.180557,3.692928e+09,0.510000,7.000000


In [78]:
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

In [79]:
merged_df['sentiment'] = merged_df['headline'].apply(analyze_sentiment)

KeyError: 'headline'

In [39]:
# Save the sentimented dataset to a CSV file
merged_df.to_csv('sentimented_df.csv', index=False)

In [40]:
merged_df.head()

,headline,url,publisher,date,stock,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Company,sentiment
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,80.837502,82.937500,80.807503,82.875000,80.843407,137250400,0.0,0.0,Apple,0.0
1,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,122.225502,124.432503,121.856499,124.150002,124.150002,66128000,0.0,0.0,Amazon,0.0
2,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,70.658501,72.252502,70.300003,71.919502,71.837753,34698000,0.0,0.0,Google,0.0
3,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,226.710007,231.350006,225.309998,230.770004,230.296753,16750400,0.0,0.0,Meta,0.0
4,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,182.619995,187.729996,182.009995,187.199997,180.403564,39893600,0.0,0.0,Microsoft,0.0


In [41]:
# Calculate daily stock returns
merged_df['Daily Return'] = merged_df['Close'].pct_change() * 100

In [42]:
merged_df.head()

,headline,url,publisher,date,stock,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Company,sentiment,Daily Return
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,80.837502,82.937500,80.807503,82.875000,80.843407,137250400,0.0,0.0,Apple,0.0,NaN
1,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,122.225502,124.432503,121.856499,124.150002,124.150002,66128000,0.0,0.0,Amazon,0.0,49.803923
2,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,70.658501,72.252502,70.300003,71.919502,71.837753,34698000,0.0,0.0,Google,0.0,-42.070478
3,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,226.710007,231.350006,225.309998,230.770004,230.296753,16750400,0.0,0.0,Meta,0.0,220.872638
4,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,182.619995,187.729996,182.009995,187.199997,180.403564,39893600,0.0,0.0,Microsoft,0.0,-18.880273


In [43]:
# Aggregate sentiments by date
daily_sentiment = merged_df.groupby('date')['sentiment'].mean().reset_index()
daily_sentiment.columns = ['date', 'Average Sentiment']

In [45]:
daily_sentiment.round(3).head()

,date,Average Sentiment
0,2011-04-27,0.000
1,2011-04-28,0.068
2,2011-04-29,0.167
3,2011-05-02,0.000
4,2011-05-03,0.000


In [46]:
# Merge sentiment data with stock data
daily_df = pd.merge(merged_df[['date', 'Daily Return']], daily_sentiment, on='date', how='inner')

In [47]:
daily_df.head()

,date,Daily Return,Average Sentiment
0,2020-06-05,NaN,0.10836
1,2020-06-05,49.803923,0.10836
2,2020-06-05,-42.070478,0.10836
3,2020-06-05,220.872638,0.10836
4,2020-06-05,-18.880273,0.10836


In [48]:
# Calculate correlation
correlation = daily_df[['Average Sentiment', 'Daily Return']].corr().iloc[0, 1]
print(f'Pearson correlation coefficient: {correlation}')

Pearson correlation coefficient: 0.000577290186737864
